In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://books.toscrape.com/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

books = []

for book in soup.find_all('article', class_="product_pod"):
    title = book.find('h3').find('a')['title']
    rating = book.find('p')['class'][1]
    price = book.find(class_='price_color').text[2:]  
    availability = "In stock" if book.find(class_='instock availability').text.strip()=="In stock" else "Out of stock"
    
    book_data = {
        "title": title,
        "rating": rating, 
        "price": price,
        "availability": availability
    }
    
    books.append(book_data)
    
df = pd.DataFrame(books)
df['title'] = df['title'].astype('str')
df['rating'] = df['rating'].astype('str')
df['price'] = df['price'].astype('float')
df['availability'] = df['availability'].astype('str')
# Dictionary mapping rating words to numbers 
rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5,
    "Six": 6,
    "Seven": 7,
    "Eight": 8, 
    "Nine": 9,
    "Ten": 10
}
# Apply mapping to replace text with numbers
df["rating"] = df["rating"].map(rating_map)

In [31]:
# Display dataframe
df.head()

,title,rating,price,availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.10,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         20 non-null     object 
 1   rating        20 non-null     int64  
 2   price         20 non-null     float64
 3   availability  20 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 772.0+ bytes


In [26]:
# !pip install snowflake-connector-python
# !pip install sqlalchemy

In [36]:
# Step 2: Data Storage with Snowflake
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine  
import pandas as pd
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine


# Snowflake connection
engine = create_engine(URL(
    account = "db68116.ap-south-1.aws",
    user = "DEVBLEND360", 
    password = "Dev@942065",
    database = "CODINGCHALLENGE",
    schema = "PUBLIC",
    warehouse = "COMPUTE_WH"
    ))

# Append dataframe to Snowflake table
df.to_sql('books', engine, if_exists='replace', index=False)

print("Books data loaded to Snowflake")


Books data loaded to Snowflake


,title,rating,price,availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.10,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.60,In stock
9,The Black Maria,1,52.15,In stock
